# Analysis of Geographic and Demographic Data To Determine New Starbucks Location

We are looking to find the next location for a Starbucks Reserve Roastery. We'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
from geopy.geocoders import Nominatim
from IPython.display import IFrame

In [2]:
# Load and preview Starbucks locations in California
starbucks_data = pd.read_csv("C:/Users/hzerr/geopan/Starbucks/data/starbucks_locations.csv")
starbucks_data.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


### Exploratory Data Analysis

In [3]:
starbucks_data.isnull().sum()

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64

In [4]:
starbucks_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2821 entries, 0 to 2820
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store Number  2821 non-null   object 
 1   Store Name    2821 non-null   object 
 2   Address       2821 non-null   object 
 3   City          2821 non-null   object 
 4   Longitude     2816 non-null   float64
 5   Latitude      2816 non-null   float64
dtypes: float64(2), object(4)
memory usage: 132.4+ KB


There are 5 rows missing the longitude and latitude. All other columns contain all necessary data.

In [5]:
starbucks_data[starbucks_data.isnull().any(axis=1)]

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


The stores with null values are located in Berkeley. Missing values for latitude and longitude will be filled in.

In [6]:
missing_data = starbucks_data[starbucks_data.isnull().any(axis=1)]

### Data Cleaning

In [7]:
# Instantiate geolocator
geolocator = Nominatim(user_agent="zhogan")

for idx, row in missing_data.iterrows():
    location = geolocator.geocode(row["Address"])

    if location:
        # Update longitude and latitude in the DataFrame
        missing_data.loc[idx, 'Longitude'] = [location.longitude]
        missing_data.loc[idx, 'Latitude'] = [location.latitude]
    else:
        print(f"Latitude and longitude for {missing_data.iloc[addy]['Address']} not found")

missing_data

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,-122.268230,37.868839
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,-122.280013,37.891477
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,-122.269679,37.880907
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,-122.259406,37.855903
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,-122.266095,37.870253


In [8]:
# Update original DataFrame with new values for latitude and longitude
starbucks_data.update(missing_data)

starbucks_data[starbucks_data["City"]=="Berkeley"]

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,-122.268230,37.868839
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,-122.280013,37.891477
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,-122.269679,37.880907
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,-122.259406,37.855903
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,-122.266095,37.870253


In [9]:
starbucks_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2821 entries, 0 to 2820
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store Number  2821 non-null   object 
 1   Store Name    2821 non-null   object 
 2   Address       2821 non-null   object 
 3   City          2821 non-null   object 
 4   Longitude     2821 non-null   float64
 5   Latitude      2821 non-null   float64
dtypes: float64(2), object(4)
memory usage: 132.4+ KB


Now all of the Starbucks data is complete.

### Data Visualization

In [10]:
# Create a base map
map1 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Add a marker for each Berkeley location
for idx, row in starbucks_data.iterrows():
    Marker(location=[starbucks_data.iloc[idx]['Latitude'], starbucks_data.iloc[idx]['Longitude']]).add_to(map1)

map1.save("map1.png")
map1.save("map1.html")

# Display the HTML map using IFrame
IFrame(src="map1.html", width=900, height=600)

All Berkeley locations appear to be correct. Next, a new file will be read in that contains the county boundaries for California.

In [11]:
CA_counties = gpd.read_file("C:/Users/hzerr/geopan/Starbucks/data/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties = CA_counties.to_crs(epsg=4326)
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


### Data Mining

Next, data frames will be created that contain population, income and age.

In [12]:
CA_pop = pd.read_csv("C:/Users/hzerr/geopan/Starbucks/data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("C:/Users/hzerr/geopan/Starbucks/data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("C:/Users/hzerr/geopan/Starbucks/data/CA_county_median_age.csv", index_col="GEOID")

In [13]:
CA_pop.head()

,population
GEOID,
6001,1666753
6003,1101
6005,39383
6007,231256
6009,45602


In [14]:
CA_high_earners.head()

,high_earners
GEOID,
6001,145696
6003,30
6005,1220
6007,6860
6009,2046


The high_earners data frame contains the number of people in each county who earn $150,000 or more annually.

In [15]:
CA_median_age.head()

,median_age
GEOID,
6001,37.3
6003,44.9
6005,50.6
6007,36.9
6009,51.6


In [16]:
# Join all data frames
CA_counties = CA_counties.merge(CA_pop, on="GEOID").merge(CA_high_earners, on="GEOID").merge(CA_median_age, on="GEOID")

In [17]:
CA_counties.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5


In [18]:
# Create a density column
CA_counties["density"] = round(CA_counties["population"] / CA_counties["area_sqkm"], 2)

In [19]:
CA_counties["density"].head()

0      1.20
1    598.38
2     45.50
3     16.98
4    148.79
Name: density, dtype: float64

Next, counties selected will be places that have a population density of at least 300 per square kilometer, a median age less than 40 and at least 200,000 high income earners.

In [20]:
candidate_counties = CA_counties[(CA_counties["density"] >= 200) & (CA_counties["median_age"] < 40) & (CA_counties["high_earners"] >= 100000)]

In [21]:
len(candidate_counties)

7

There are 7 potential counties for the new Starbucks Reserve Roastery location.

In [22]:
# Convert Starbucks data frame to GDF before joining
starbucks_gdf = gpd.GeoDataFrame(starbucks_data, geometry=gpd.points_from_xy(starbucks_data.Longitude, starbucks_data.Latitude))
starbucks_gdf = starbucks_gdf.set_crs(epsg=4326)

candidate_stores = gpd.sjoin(starbucks_gdf, candidate_counties, how="inner")

In [23]:
candidate_stores

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry,index_right,GEOID,name,area_sqkm,population,high_earners,median_age,density
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.23
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.23
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87,POINT (-117.75000 33.87000),47,6059,Orange County,2455.308632,3185968,233459,37.5,1297.58
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79,POINT (-122.28000 37.79000),41,6001,Alameda County,2127.222169,1666753,145696,37.3,783.53
5,6479-62999,Park & Central Alameda,1364 Park Street Alameda CA,Alameda,-122.24,37.76,POINT (-122.24000 37.76000),41,6001,Alameda County,2127.222169,1666753,145696,37.3,783.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2806,9384-94268,Canoga & Oxnard,"5960 N. Canoga Ave, #1 Woodland Hills CA",Woodland Hills,-118.60,34.18,POINT (-118.60000 34.18000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.23
2807,18869-179006,Mulholland & Calabasas,23335 Mulholland Drive Woodland Hills CA,Woodland Hills,-118.63,34.16,POINT (-118.63000 34.16000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.23
2808,11603-104000,Topanga Canyon & Dumetz,4900 Topanga Canyon Blvd Woodland Hills CA,Woodland Hills,-118.61,34.16,POINT (-118.61000 34.16000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.23
2809,5450-1464,Yorba Linda & Village Center Dr,"20355 Yorba Linda Blvd, Valley View Shopping C...",Yorba Linda,-117.78,33.89,POINT (-117.78000 33.89000),47,6059,Orange County,2455.308632,3185968,233459,37.5,1297.58


There are 1,719 stores in counties that meet the criteria for a new store.

### Conclusion

In [24]:
map2 = folium.Map(location=[37,-120], zoom_start=6)

# Show selected store locations
for i, j in candidate_stores.iterrows():
    Marker(
        location=[j["Latitude"], j["Longitude"]],
        popup=f'{j["Address"]}'
    ).add_to(map2)

map2.save("map2.png")
map2.save("map2.html")

# Display the HTML map using IFrame
IFrame(src="map1.html", width=900, height=600)

On the map, 1,719 locations meet the requirements for a new store based on income, age and population density. Clicking on each marker allows us to see the actual store address.